# GraphRAG Patterns

Let's explore two key GraphRAG patterns:
1. Customer Context Retrieval
2. Product Knowledge Base

In [ ]:
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os

load_dotenv()
driver = GraphDatabase.driver(
    os.getenv('NEO4J_URI'),
    auth=(os.getenv('NEO4J_USERNAME'), os.getenv('NEO4J_PASSWORD'))
)

## Pattern 1: Customer Context Retrieval

This pattern helps us understand a customer's history and context when they ask a question.

In [ ]:
def get_customer_context(customer_id):
    with driver.session() as session:
        result = session.run("""
        MATCH (c:Customer {customer_id: $customer_id})
        
        // Get recent orders
        OPTIONAL MATCH (c)-[:PLACED_ORDER]->(o:Order)-[:CONTAINS]->(p:Product)
        
        // Get support cases
        OPTIONAL MATCH (c)-[:HAS_CASE]->(d:Document {type: 'support_case'})
        
        // Get product manuals for purchased products
        OPTIONAL MATCH (p)-[:HAS_MANUAL]->(m:Document {type: 'manual'})
        
        RETURN c.name as customer,
               collect(DISTINCT p.name) as products,
               collect(DISTINCT d.content) as support_cases,
               collect(DISTINCT m.content) as manuals
        """, customer_id=customer_id)
        
        return result.single()

# Get context for John Doe
context = get_customer_context('C1')
print(f"Customer: {context['customer']}")
print(f"Products: {context['products']}")
print(f"\nSupport Cases: {len(context['support_cases'])}")
print(f"Product Manuals: {len(context['manuals'])}")

## Pattern 2: Product Knowledge Base

This pattern helps us find relevant product information and related support cases.

In [ ]:
def query_product_knowledge(product_id, query):
    with driver.session() as session:
        result = session.run("""
        MATCH (p:Product {product_id: $product_id})
        
        // Get product manual
        OPTIONAL MATCH (p)-[:HAS_MANUAL]->(m:Document)
        WHERE m.content CONTAINS $query
        
        // Get related support cases
        OPTIONAL MATCH (p)<-[:ABOUT]-(c:Document)
        WHERE c.content CONTAINS $query
        
        // Get similar products
        OPTIONAL MATCH (p)-[:IN_CATEGORY]->(cat:Category)<-[:IN_CATEGORY]-(sp:Product)
        WHERE p <> sp
        
        RETURN p.name as product,
               collect(DISTINCT m.content) as manual_matches,
               collect(DISTINCT c.content) as case_matches,
               collect(DISTINCT sp.name) as similar_products
        """, product_id=product_id, query=query)
        
        return result.single()

# Query about laptop power issues
knowledge = query_product_knowledge('P1', 'power')
print(f"Product: {knowledge['product']}\n")
print("Manual Matches:")
for match in knowledge['manual_matches']:
    print(f"- {match[:100]}...\n")
print("Support Case Matches:")
for match in knowledge['case_matches']:
    print(f"- {match[:100]}...\n")
print(f"Similar Products: {knowledge['similar_products']}")

## Combining Patterns

Let's use both patterns to answer a customer question.

In [ ]:
def answer_customer_question(customer_id, question):
    # First, get customer context
    context = get_customer_context(customer_id)
    
    # For each product they own, search knowledge base
    relevant_info = []
    with driver.session() as session:
        result = session.run("""
        MATCH (c:Customer {customer_id: $customer_id})
        MATCH (c)-[:PLACED_ORDER]->(:Order)-[:CONTAINS]->(p:Product)
        RETURN DISTINCT p.product_id as product_id
        """, customer_id=customer_id)
        
        for record in result:
            knowledge = query_product_knowledge(record['product_id'], question)
            if knowledge['manual_matches'] or knowledge['case_matches']:
                relevant_info.append(knowledge)
    
    return {
        'customer': context['customer'],
        'products': context['products'],
        'relevant_info': relevant_info
    }

# Answer a question about power issues
answer = answer_customer_question('C1', 'power issues')
print(f"Finding answer for {answer['customer']}")
print(f"Products owned: {answer['products']}\n")
print("Relevant Information:")
for info in answer['relevant_info']:
    print(f"\nProduct: {info['product']}")
    if info['manual_matches']:
        print("From Manual:")
        print(info['manual_matches'][0][:200])
    if info['case_matches']:
        print("\nFrom Support Cases:")
        print(info['case_matches'][0][:200])

## Next Steps

Now let's see how we can enhance this with a memory graph to track interactions!